<a href="https://colab.research.google.com/github/AmiraBelkis/tsp-ant-colony/blob/AmiraBelkis-Param-1/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import Parser
instance = Parser.TSPInstance('data/data_101.tsp')
instance.readData()
data = np.copy(instance.data)

ModuleNotFoundError: ignored

## Helper methods

In [3]:
def norm_random():
    return np.random.randint(100)/100

This function represents the amount of attraciveness of a path against the pheromones put in it 
$({\tau^{\alpha}_x}_y)\times({\zeta^{\beta}_x}_y)$


In [4]:
def attract_pherm(x,y,alpha,beta,pheromone,data):
    return (pheromone[x][y]**alpha)*((1/data[x][y])**beta)

In [5]:
def show_paths(ants):
    print('completed iteration : ')
    i = 1
    for ant in ants :
        print('ant',i,':',ant.path,', cost :',ant.cost)
        i+=1

## Class of ant 

In [6]:
class Ant:
    def __init__(self,begin):
        self.path = [begin]
        self.cost = 0.0
    
    #finding next node to visit in stochastic way
    def next_node(self,data=[],pheromone=[],alpha=0.5,beta=0.5,random_rate=0.3):
        next_node = -1
        #case of an ant following the exploit method
        if(norm_random()>=random_rate):
            allowed_nodes = []
            sum_of_probs = 0.0
            #get all possible nodes
            for i in range(len(data)):
                if not (i in self.path): 
                    allowed_nodes.append(i)
                    sum_of_probs += attract_pherm(x= self.path[-1] , y= i , alpha=alpha,beta=beta,pheromone=pheromone,data=data)
            #counting probabilities
            max_prob = 0

            for node in allowed_nodes:
                prob_xy = (attract_pherm(x= self.path[-1] , y= node , alpha=alpha,beta=beta,pheromone=pheromone,data=data))/sum_of_probs
             
                if prob_xy >= max_prob : 
                    next_node = node
                    max_prob = prob_xy
           
        #case of an ant following the exploring method
        else : 
            node = 0 
            while(node in self.path):
                node +=1
            next_node = node

        #adding cost 
        self.cost+= data[self.path[-1]][next_node]

        if(len(self.path)==len(data)):
            self.cost+= data[self.path[-1]][0]
            
        self.path.append(next_node)
    
    #updating pheromone 
    def update_pheromone(self, pheromone,q):
        for i in range(len(self.path)-1):
            pheromone[self.path[i]][self.path[i+1]]+= q/self.cost
            pheromone[self.path[i+1]][self.path[i]]+= q/self.cost
        new_ph = pheromone
        return new_ph
    
    #reset path
    def reset_path(self):
        self.cost = 0.0
        self.path = [self.path[0]]

## Initializing procedures

In [7]:
def initialize_pheromone_matrix(data):
    m = np.ones(data.shape)
    for i in range(len(m)):
        m[i][i]= 0
    return m 

In [8]:
def initialize_colony(ants_number, data):
    ants= []
    begin = np.random.randint(len(data))
    for i in range(ants_number):
        ants.append(Ant(begin))
    return ants , begin

## Ant Colony Algorithm 

In [12]:
def AntColony(data,ants_number = 4, alpha = 0.7 , beta = 0.4 , evaporate_rate = 0.1,random_rate=0.5 , show_history = False):
    assert ants_number >=2 
    path = []
    cost = 0.0
    stop = False 
    pheromone = initialize_pheromone_matrix(data)
    ants , begin = initialize_colony(ants_number=ants_number, data=data)
    while (not stop) : 
        #first : visit nodes with ants
        for ant in ants:
            #looking for a cycle
            while (len(ant.path) != len(data)):
                ant.next_node(data=data,pheromone=pheromone, alpha=alpha, beta = beta,random_rate=random_rate)
        
        #second : evaporation 
        pheromone=np.dot(pheromone,1-evaporate_rate)
        
        #last : update the pheremone in nodes 
        for ant in ants:
            pheromone = ant.update_pheromone(pheromone=pheromone,q=10)
        
        #checking the stop condition
        path = ants[0].path
        stop = True
        for ant in ants[1:]:
            stop = stop and np.array_equal(path,ant.path)
        random_rate = random_rate / 10
        
        #an optional feature to print the history of paths
        if show_history :  
            show_paths(ants=ants)

        #reseting paths 
        for ant in ants :
            if not stop : ant.reset_path()
    else :
        return ants[0].path , ants[0].cost

In [13]:
p,c = AntColony(data=data,ants_number=4,alpha = 0.5,beta = 0.1,evaporate_rate=0.1,random_rate=0.7,show_history=False)
print(p , c)

NameError: ignored

## Running algorithm multiple times to get lower cost

In [ ]:
p = [] 
c = np.inf
for i in range(20): 
    path , cost = AntColony(data=data,ants_number=5,alpha = 0.5,beta = 0.1,evaporate_rate=0.1,random_rate=0.7,show_history=False)
    if(cost < c ):
        c = cost 
        p = path

In [ ]:
print(p , c)

#  Create dataset to compare parametres

In [15]:
import time
import random
# writing data result to a CSV file to use it in ANOVA 
import csv 

# field names                                                    
fields = ['data_set','Ants_number', 'Alpha', 'Beta' , 'Evaporate_rate', 'Random_rate', 'nbr_iterations', 'cost', 'time'] 
    
# name of csv file 
fileURL = "https://raw.githubusercontent.com/AmiraBelkis/tsp-ant-colony/AmiraBelkis-Param-1/data.csv"    

# writing to csv file 
with open(fileURL, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)       
    # writing the fields 
    csvwriter.writerow(fields) 
    datasetList = ['data/data_101.tsp']
    for dataset in datasetList: 
        instance = Parser.TSPInstance(dataset)
        instance.readData()
        data = np.copy(instance.data)
        # ants_number loop
        for i1 in range(100):
            Ants_number = random.randint(2,15)
            # alpha 
            Alpha = random.randint(1,50)
            # beta 
            Beta = random.randint(1,50)
            # evaporate_rate 
            Evaporate_rate = random.randint(5,95)/100
            # random_rate 
            Random_rate = 0.5 #random.randint(5,95)/100
            # nbr_iterations loop
            nbr_iterations = random.randint(1,10)
            p = [] 
            c = np.inf
            start_time = time.time()
            for i in range(nbr_iterations): 
                path , cost = AntColony(data ,Ants_number ,Alpha ,Beta ,Evaporate_rate ,Random_rate ,show_history=False)                                    
                if(cost < c ):
                    c = cost 
                    p = path
            end_time = time.time()
            csvwriter.writerow([dataset,Ants_number, Alpha,Beta ,Evaporate_rate, Random_rate, nbr_iterations, c ,  end_time-start_time])
  

FileNotFoundError: ignored